In [1]:
from PIL import Image
import numpy as np
import tensorflow as tf
import os
from tensorpack import *
from tensorpack.tfutils.symbolic_functions import *
from tensorpack.tfutils.summary import *
from tflearn.layers.conv import global_avg_pool
from tensorflow.contrib.layers import batch_norm, flatten
from tensorflow.contrib.layers import xavier_initializer
from tensorflow.contrib.framework import arg_scope

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
height = 224
width = 224

In [3]:
def conv_layer(input, filter, kernel, stride=1, layer_name="conv"):
    with tf.name_scope(layer_name):
        network = tf.layers.conv2d(inputs=input, use_bias=False, filters=filter, kernel_size=kernel, strides=stride, padding='SAME')
        return network

def Global_Average_Pooling(x, stride=1):
    return global_avg_pool(x, name='Global_avg_pooling')



def Batch_Normalization(x, training, scope):
    with arg_scope([batch_norm],
                   scope=scope,
                   updates_collections=None,
                   decay=0.9,
                   center=True,
                   scale=True,
                   zero_debias_moving_mean=True) :
        return tf.cond(training,
                       lambda : batch_norm(inputs=x, is_training=training, reuse=None),
                       lambda : batch_norm(inputs=x, is_training=training, reuse=True))

def Drop_out(x, rate, training) :
    return tf.layers.dropout(inputs=x, rate=rate, training=training)

def Relu(x):
    return tf.nn.relu(x)

def Average_pooling(x, pool_size=[2,2], stride=2, padding='VALID'):
    return tf.layers.average_pooling2d(inputs=x, pool_size=pool_size, strides=stride, padding=padding)


def Max_Pooling(x, pool_size=[3,3], stride=2, padding='VALID'):
    return tf.layers.max_pooling2d(inputs=x, pool_size=pool_size, strides=stride, padding=padding)

def Concatenation(layers) :
    return tf.concat(layers, axis=3)

def Linear(x,units,name) :
    return tf.layers.dense(inputs=x, units=units, name=name)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
class VGG():
    def __init__(self, x,training):
        self.training = training
        self.model = self.VGG(x)
            
    def VGG(self, x):
        x1 = conv_layer(x, filter=64, kernel=[3,3],layer_name='conv1',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch1')
        x1 = Relu(x1)
        x1 = conv_layer(x1, filter=64, kernel=[3,3],layer_name='conv2',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch2')
        x1 = Relu(x1)
        x1 = Max_Pooling(x1,pool_size=[2,2],stride=2)
        
        x1 = conv_layer(x1, filter=128, kernel=[3,3],layer_name='conv3',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch3')
        x1 = Relu(x1)
        x1 = conv_layer(x1, filter=128, kernel=[3,3],layer_name='conv4',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch4')
        x1 = Relu(x1)
        x1 = Max_Pooling(x1,pool_size=[2,2],stride=2)
        
        x1 = conv_layer(x1, filter=256, kernel=[3,3],layer_name='conv5',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch5')
        x1 = Relu(x1)
        x1 = conv_layer(x1, filter=256, kernel=[3,3],layer_name='conv6',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch6')
        x1 = Relu(x1)
        x1 = conv_layer(x1, filter=256, kernel=[3,3],layer_name='conv7',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch7')
        x1 = Relu(x1)
        x1 = Max_Pooling(x1,pool_size=[2,2],stride=2)
        
        x1 = conv_layer(x1, filter=512, kernel=[3,3],layer_name='conv8',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch8')
        x1 = Relu(x1)
        x1 = conv_layer(x1, filter=512, kernel=[3,3],layer_name='conv9',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch9')
        x1 = Relu(x1)
        x1 = conv_layer(x1, filter=512, kernel=[3,3],layer_name='conv10',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch10')
        x1 = Relu(x1)
        x1 = Max_Pooling(x1,pool_size=[2,2],stride=2)
        
        x1 = conv_layer(x1, filter=512, kernel=[3,3],layer_name='conv11',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch11')
        x1 = Relu(x1)
        x1 = conv_layer(x1, filter=512, kernel=[3,3],layer_name='conv12',stride=1)
        x1 = Batch_Normalization(x1, training=self.training, scope='batch12')
        x1 = Relu(x1)
        x1 = conv_layer(x1, filter=512, kernel=[3,3],layer_name='conv13',stride=1)
        
        x1 = Global_Average_Pooling(x1)
        x1 = flatten(x1)

        clf_tensor = Linear(x1,1,"clf")
        
        
        return tf.reshape(clf_tensor,[-1])

In [5]:
img_channels = 1
x = tf.placeholder(tf.float32, shape=[None, height, width,img_channels])
training_flag = tf.placeholder(tf.bool)
logits = VGG(x=x,training=training_flag).model

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


In [6]:
saver = tf.train.Saver(tf.global_variables())
sess = tf.Session()
ckpt = tf.train.get_checkpoint_state('./model/disc_classification_model/')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model/disc_classification_model/model.ckpt


In [7]:

def crop_margin_PIL(image):
    im = image
    np_img = np.asarray(im)
    mean_pix = np.mean(np_img)
    pix = im.load()
    height, width = im.size

    c_x, c_y = (int(height / 2), int(width / 2))

    for y in range(c_y):
        if sum(pix[c_x, y]) > mean_pix:
            left = (c_x, y)
            break;

    for x in range(c_x):
        if sum(pix[x, c_y]) > mean_pix:
            up = (x, c_y)
            break;

    crop_img = im.crop((up[0], left[1], left[0], up[1]))

    diameter_height = up[1] - left[1]
    diameter_width = left[0] - up[0]

    crop_img = im.crop((up[0], left[1], left[0] + diameter_width, up[1] + diameter_height))

    return crop_img

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [8]:
img_dir = "/home/mediwhale/SEED/SEED_1/RETINA - SCES/SICC05/2009-02-09/CS50023/CS50023_KK00(POZ.002.JPG"


img = Image.open(img_dir)
img = crop_margin_PIL(img)
img = img.convert("L").resize((height,width))

img = np.array(img).reshape(height,width,1)
img = img / 255.

result = ""

In [10]:
feed_dict = {
                x: [img],
                training_flag : False
            }

r = sess.run(logits,feed_dict=feed_dict)
r = sigmoid(r)

In [13]:

if r[0] < 0.5 :
    result = "disc_focus_right"
else :
    result = "disc_focus_left"

In [14]:
result

'disc_focus_right'